# DNN Experiments Notebook

Train, compare, and visualize DNN solutions for corporate finance models.

**Contents:**
1. Basic Model: LR vs ER vs BR comparison
2. Risky Debt Model: BR+price with diagnostics
3. Network Architecture Comparison
4. Economic Scenario Comparison (fixed adjustment cost)

## Configuration (Single Source of Truth)

In [ ]:
# =============================================================================
# IMPORTS & PATH SETUP
# =============================================================================
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.dnn import (
    # Config
    TrainingConfig, EconomicScenario, SamplingBounds,
    # Training
    train_basic_lr, train_basic_er, train_basic_br,
    train_risky_lr, train_risky_br,
    # Evaluation
    get_eval_grids, evaluate_basic_policy, compute_moments,
    # Plotting
    plot_loss_curve, plot_loss_comparison,
)

In [ ]:
# =============================================================================
# GLOBAL SETTINGS
# =============================================================================

# --- Run Mode: Toggle between fast debugging and full precision ---
RUN_MODE = "debug"  # "debug" (30-60s) or "full" (5+ min)

# --- Logging verbosity ---
VERBOSE = False  # Set True for detailed print statements

# --- Random seed for reproducibility ---
SEED = 42

# --- Plotting defaults ---
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

In [ ]:
# =============================================================================
# TRAINING CONFIG PRESETS
# =============================================================================
# These presets control training speed/quality tradeoff.
# - n_iter: Number of training iterations (more = slower, more converged)
# - batch_size: Samples per gradient step (larger = fewer steps)
# - n_neurons: Network width (larger = more capacity, slower)
# - log_every: Logging frequency (higher = fewer log points)

if RUN_MODE == "debug":
    # Debug mode: Fast iteration for correctness checks
    CONFIG_BASIC = TrainingConfig(
        n_layers=2, n_neurons=16, n_iter=50,
        batch_size=256, learning_rate=3e-3, log_every=10, seed=SEED
    )
    CONFIG_RISKY = TrainingConfig(
        n_layers=2, n_neurons=16, n_iter=50,
        batch_size=256, learning_rate=3e-3, log_every=10, seed=SEED,
        epsilon_D_0=0.1, decay_d=0.90
    )
else:  # "full"
    # Full mode: Higher precision for publication-quality results
    CONFIG_BASIC = TrainingConfig(
        n_layers=2, n_neurons=32, n_iter=200,
        batch_size=128, learning_rate=1e-3, log_every=10, seed=SEED
    )
    CONFIG_RISKY = TrainingConfig(
        n_layers=2, n_neurons=32, n_iter=100,
        batch_size=128, learning_rate=1e-3, log_every=5, seed=SEED,
        epsilon_D_0=0.1, decay_d=0.95
    )

print(f"RUN_MODE = '{RUN_MODE}' | n_iter = {CONFIG_BASIC.n_iter} | batch = {CONFIG_BASIC.batch_size}")

In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def log(msg: str):
    """Print message only if VERBOSE is True."""
    if VERBOSE:
        print(msg)

def section_header(title: str):
    """Print a visual section header."""
    print(f"\n{'='*60}\n{title}\n{'='*60}")

def summary_line(name: str, value, fmt: str = ".4f"):
    """Print a formatted summary line."""
    print(f"  {name}: {value:{fmt}}")

## 1. Basic Model: LR vs ER vs BR Comparison

In [ ]:
section_header("1. BASIC MODEL COMPARISON")

# Define scenario locally for this section
scenario_basic = EconomicScenario(name="baseline")

# Train all three methods
log("Training LR...")
history_lr = train_basic_lr(scenario_basic, CONFIG_BASIC)

log("Training ER...")
history_er = train_basic_er(scenario_basic, CONFIG_BASIC)

log("Training BR...")
history_br = train_basic_br(scenario_basic, CONFIG_BASIC)

# Summary
print("Final losses:")
summary_line("LR (neg. reward)", history_lr['loss_LR'][-1])
summary_line("ER (Euler resid)", history_er['loss_ER'][-1], ".6f")
summary_line("BR critic", history_br['loss_BR_critic'][-1], ".6f")
summary_line("BR actor", history_br['loss_BR_actor'][-1])

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

ax = axes[0]
ax.plot(history_lr["iteration"], history_lr["loss_LR"], 'b-', linewidth=2)
ax.set_xlabel("Iteration"); ax.set_ylabel("Loss LR"); ax.set_title("Lifetime Reward")
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(history_er["iteration"], history_er["loss_ER"], 'g-', linewidth=2)
ax.set_xlabel("Iteration"); ax.set_ylabel("Loss ER"); ax.set_title("Euler Residual")
ax.set_yscale('log'); ax.grid(True, alpha=0.3)

ax = axes[2]
ax.plot(history_br["iteration"], history_br["loss_BR_critic"], 'b-', label="Critic", linewidth=2)
ax.plot(history_br["iteration"], history_br["loss_BR_actor"], 'r--', label="Actor", linewidth=2)
ax.set_xlabel("Iteration"); ax.set_ylabel("Loss"); ax.set_title("BR Actor-Critic")
ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Risky Debt Model: BR+Price with Diagnostics

In [ ]:
section_header("2. RISKY DEBT MODEL (BR+PRICE)")

# Define scenario locally
scenario_risky = EconomicScenario(name="baseline")

# Train BR+price
log("Training Risky Debt BR+price...")
history_risky = train_risky_br(scenario_risky, CONFIG_RISKY)

print("Final losses:")
summary_line("Critic", history_risky['loss_critic'][-1], ".6f")
summary_line("Actor", history_risky['loss_actor'][-1])
summary_line("Price", history_risky['loss_price'][-1], ".6f")
summary_line("epsilon_D", history_risky['epsilon_D'][-1], ".4f")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

ax = axes[0]
ax.plot(history_risky["iteration"], history_risky["loss_critic"], 'b-', linewidth=2)
ax.set_xlabel("Iteration"); ax.set_ylabel("Critic Loss"); ax.set_title("Bellman Residual")
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(history_risky["iteration"], history_risky["loss_price"], 'g-', linewidth=2)
ax.set_xlabel("Iteration"); ax.set_ylabel("Price Loss"); ax.set_title("Pricing Residual")
ax.grid(True, alpha=0.3)

ax = axes[2]
ax.plot(history_risky["iteration"], history_risky["epsilon_D"], 'purple', linewidth=2)
ax.set_xlabel("Iteration"); ax.set_ylabel("epsilon_D"); ax.set_title("Default Smoothing")
ax.set_yscale('log'); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Gradient diagnostics
print("\nGradient Diagnostics:")
summary_line("grad_norm_policy", history_risky['grad_norm_policy'][-1])
summary_line("share_V_tilde<0", history_risky['share_v_tilde_negative'][-1]*100, ".1f%%")
if history_risky["grad_norm_policy"][-1] == 0:
    print("  WARNING: Policy gradient is zero!")

## 3. Network Architecture Comparison

In [ ]:
section_header("3. NETWORK ARCHITECTURE COMPARISON")

# Define scenario locally
scenario_arch = EconomicScenario(name="baseline")

# Small network
config_small = TrainingConfig(
    n_layers=2, n_neurons=16, n_iter=CONFIG_BASIC.n_iter,
    batch_size=CONFIG_BASIC.batch_size, learning_rate=CONFIG_BASIC.learning_rate,
    log_every=CONFIG_BASIC.log_every, seed=SEED
)

# Large network
config_large = TrainingConfig(
    n_layers=3, n_neurons=64, n_iter=CONFIG_BASIC.n_iter,
    batch_size=CONFIG_BASIC.batch_size, learning_rate=CONFIG_BASIC.learning_rate,
    log_every=CONFIG_BASIC.log_every, seed=SEED
)

log("Training 2×16 network...")
history_small = train_basic_lr(scenario_arch, config_small)

log("Training 3×64 network...")
history_large = train_basic_lr(scenario_arch, config_large)

print("Final LR losses:")
summary_line("2×16 network", history_small['loss_LR'][-1])
summary_line("3×64 network", history_large['loss_LR'][-1])

In [ ]:
fig = plot_loss_comparison(
    [history_small, history_large],
    ["2×16 network", "3×64 network"],
    title="Network Architecture Comparison"
)
plt.show()

## 4. Economic Scenario Comparison (Fixed Adjustment Cost)

In [ ]:
section_header("4. FIXED ADJUSTMENT COST COMPARISON")

# Define scenarios locally
scenario_low = EconomicScenario.from_overrides("low_adj_cost", cost_convex=0.01, cost_fixed=0.0)
scenario_high = EconomicScenario.from_overrides("high_adj_cost", cost_convex=0.01, cost_fixed=0.5)

print(f"Low adj cost : cost_fixed = {scenario_low.params.cost_fixed}")
print(f"High adj cost: cost_fixed = {scenario_high.params.cost_fixed}")

In [ ]:
# Train both scenarios
log("Training low adj cost...")
history_low = train_basic_br(scenario_low, CONFIG_BASIC)

log("Training high adj cost...")
history_high = train_basic_br(scenario_high, CONFIG_BASIC)

print("\nFinal BR critic losses:")
summary_line("Low adj cost", history_low['loss_BR_critic'][-1], ".6f")
summary_line("High adj cost", history_high['loss_BR_critic'][-1], ".6f")

In [ ]:
# Get evaluation grids from scenario bounds
k_grid, z_grid, _ = get_eval_grids(scenario_low, n_k=50, n_z=10)

eval_low = evaluate_basic_policy(history_low["_policy_net"], k_grid, z_grid)
eval_high = evaluate_basic_policy(history_high["_policy_net"], k_grid, z_grid)

In [ ]:
# Plot policy comparison (k' and I/k vs k at median z)
z_idx = len(z_grid) // 2

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.plot(k_grid, eval_low["k_next"][:, z_idx], 'b-', label="Low adj cost", linewidth=2)
ax.plot(k_grid, eval_high["k_next"][:, z_idx], 'r--', label="High adj cost", linewidth=2)
ax.plot(k_grid, k_grid, 'k:', alpha=0.5, label="45° line")
ax.set_xlabel("k"); ax.set_ylabel("k'")
ax.set_title(f"Capital Policy (at z = {z_grid[z_idx]:.2f})")
ax.legend(); ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(k_grid, eval_low["I_k"][:, z_idx], 'b-', label="Low adj cost", linewidth=2)
ax.plot(k_grid, eval_high["I_k"][:, z_idx], 'r--', label="High adj cost", linewidth=2)
ax.axhline(0, color='k', linestyle=':', alpha=0.5)
ax.set_xlabel("k"); ax.set_ylabel("I/k")
ax.set_title(f"Investment Rate (at z = {z_grid[z_idx]:.2f})")
ax.legend(); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot policy vs ln(z) at median k
k_idx = len(k_grid) // 2
ln_z_grid = np.log(z_grid)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.plot(ln_z_grid, eval_low["k_next"][k_idx, :], 'b-', label="Low adj cost", linewidth=2)
ax.plot(ln_z_grid, eval_high["k_next"][k_idx, :], 'r--', label="High adj cost", linewidth=2)
ax.axhline(k_grid[k_idx], color='k', linestyle=':', alpha=0.5)
ax.set_xlabel("ln(z)"); ax.set_ylabel("k'")
ax.set_title(f"Capital Policy (at k = {k_grid[k_idx]:.2f})")
ax.legend(); ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(ln_z_grid, eval_low["I_k"][k_idx, :], 'b-', label="Low adj cost", linewidth=2)
ax.plot(ln_z_grid, eval_high["I_k"][k_idx, :], 'r--', label="High adj cost", linewidth=2)
ax.axhline(0, color='k', linestyle=':', alpha=0.5)
ax.set_xlabel("ln(z)"); ax.set_ylabel("I/k")
ax.set_title(f"Investment Rate (at k = {k_grid[k_idx]:.2f})")
ax.legend(); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

Demonstrated training and visualization for:
1. Basic model with LR, ER, BR methods
2. Risky debt with BR+price and gradient diagnostics
3. Network architecture comparison
4. Economic scenario comparison (fixed adjustment cost)

To switch from debug to full precision: change `RUN_MODE = "full"` at top.

In [ ]:
print(f"\n{'='*60}\nNotebook complete. RUN_MODE = '{RUN_MODE}'\n{'='*60}")